In [ ]:
import json
import openai
import csv
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
openai.api_key = ''

In [ ]:
from tenacity import retry, stop_after_attempt, wait_random_exponential

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def generate_gpt35_response_base(user_input):
    """
    Query OpenAI GPT-3.5 for the specific key and get back a response
    :type user_text: str the user's text to query for
    :type print_output: boolean whether or not to print the raw output JSON
    """
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
            {"role": "system", "content": "Give your opinion."},
            {"role": "user", "content": user_input},
        ],
    temperature=0,
    max_tokens=100,
    request_timeout=15
    )
    
    message = response.choices[0].message.content
    return message

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def generate_gpt35_response_conditional(user_input):
    """
    Query OpenAI GPT-3.5 for the specific key and get back a response
    :type user_text: str the user's text to query for
    :type print_output: boolean whether or not to print the raw output JSON
    """
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
            {"role": "system", "content": "You are behaving as a specified demographic group."},
            {"role": "user", "content": user_input},
        ],
    temperature=0,
    max_tokens=100,
    request_timeout=15
    )
    
    message = response.choices[0].message.content
    return message

In [ ]:
data = pd.read_csv('regression_data_prelim.csv')
data.head()

In [ ]:
with open('./prompts/groupharm.txt', 'r') as f:
    i = 1
    myf = f.read()
    with open("groupharm-results.csv", 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["claim", "gender", "true_label", "gpt_output"])
        for index, row in data.iterrows():
            claim = row['claim']
            gender = row['Gender']
            true_label = row['GroupHarm']
            prompt = myf.format(claim=claim) 
            print(prompt)
            response = generate_gpt35_response_base(prompt)
            print("iteration number " + str(i) + ", response is: " + response.strip())
#             break
            i += 1
            info = [claim, gender, true_label, response.strip()]
            writer.writerow(info)

In [ ]:
genders = ['man', 'woman']
unique_claims = list(set(data['claim'].tolist()))

with open('./prompts/groupharm-conditional.txt', 'r') as f:
    i = 1
    myf = f.read()
    with open("groupharm-conditional-results.csv", 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["claim", "male_label", "female_label"])
        for c in unique_claims:
            responses = []
            for g in genders:
                prompt = myf.format(claim=c, gender=g)
                response = generate_gpt35_response_conditional(prompt)
                responses.append(response)
                print(prompt)
                print("iteration number " + str(i) + ", response is: " + response.strip())
                i += 1
            info = [c, responses[0], responses[1]]
            writer.writerow(info)

In [ ]:
harm_base = pd.read_csv('groupharm-results.csv')
harm_base_pred = harm_base['gpt_output'].tolist()
filtered = [x[:1] for x in harm_base_pred]
Counter(filtered)

In [ ]:
harm_cond = pd.read_csv('groupharm-conditional-results.csv')
harm_cond_male = harm_cond['male_gpt_output'].tolist()
harm_cond_female = harm_cond['female_gpt_output'].tolist()
harm_cond_male = [x[:1] for x in harm_cond_male]
harm_cond_female = [x[:1] for x in harm_cond_female]

print(Counter(harm_cond_male))
print(Counter(harm_cond_female))

In [ ]:
# extracting the label from the original GPT output, which is just the first character in the output
with open("groupharm-cleaned-results.csv", 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["claim", "gender", "true_label", "gpt_output", "predict_label"])
    for index, row in harm_base.iterrows():
        claim = row['claim']
        gender = row['gender']
        true_label = row['true_label']
        gpt_output = row['gpt_output']
        predict_label = gpt_output[:1]
        writer.writerow([claim, gender, true_label, gpt_output, predict_label])

In [ ]:
# extracting the label from the original GPT output, which is just the first character in the output
with open("groupharm-cleaned-conditional-results.csv", 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["claim", "male_gpt_output", "female_gpt_output", "male_label", "female_label"])
    for index, row in harm_cond.iterrows():
        claim = row['claim']
        male_gpt_output = row['male_gpt_output']
        female_gpt_output = row['female_gpt_output']
        male_label = male_gpt_output[:1]
        female_label = female_gpt_output[:1]
        writer.writerow([claim, male_gpt_output, female_gpt_output, male_label, female_label])